## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Findlay,US,41.04,-83.65,87.80,55,1,14.99,clear sky
1,1,Saint-Philippe,RE,-21.36,55.77,68.00,60,36,4.70,scattered clouds
2,2,Santona,ES,43.44,-3.46,72.00,88,90,5.82,overcast clouds
3,3,Qaanaaq,GL,77.48,-69.36,38.98,86,100,10.98,light rain
4,4,Ngunguru,NZ,-35.62,174.50,62.01,3,8,5.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Findlay,US,41.04,-83.65,87.80,55,1,14.99,clear sky
9,9,Atuona,PF,-9.80,-139.03,78.82,75,69,18.59,broken clouds
15,15,Marawi,PH,8.00,124.29,78.01,86,59,4.00,broken clouds
19,19,Shouguang,CN,36.88,118.74,78.01,71,47,3.69,scattered clouds
23,23,Kapaa,US,22.08,-159.32,86.00,58,20,11.41,few clouds
24,24,Vaini,TO,-21.20,-175.20,80.60,69,40,8.05,scattered clouds
31,31,Dingle,PH,11.00,122.67,80.74,77,98,4.90,overcast clouds
39,39,Butaritari,KI,3.07,172.79,82.60,69,100,10.94,overcast clouds
42,42,Surt,LY,31.21,16.59,78.58,74,0,7.76,clear sky
45,45,Mana,GF,5.66,-53.78,78.66,86,68,2.57,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Findlay,US,41.04,-83.65,87.80,55,1,14.99,clear sky
9,9,Atuona,PF,-9.80,-139.03,78.82,75,69,18.59,broken clouds
15,15,Marawi,PH,8.00,124.29,78.01,86,59,4.00,broken clouds
19,19,Shouguang,CN,36.88,118.74,78.01,71,47,3.69,scattered clouds
23,23,Kapaa,US,22.08,-159.32,86.00,58,20,11.41,few clouds
...,...,...,...,...,...,...,...,...,...,...
686,686,Kilinochchi,LK,9.40,80.40,78.58,83,19,10.67,few clouds
687,687,Lorengau,PG,-2.02,147.27,84.69,72,2,7.47,light rain
688,688,Bubaque,GW,11.28,-15.83,80.67,78,100,8.61,light rain
693,693,San Rafael Del Sur,NI,11.85,-86.44,82.40,78,40,4.70,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Findlay,US,87.80,clear sky,41.04,-83.65,
9,Atuona,PF,78.82,broken clouds,-9.80,-139.03,
15,Marawi,PH,78.01,broken clouds,8.00,124.29,
19,Shouguang,CN,78.01,scattered clouds,36.88,118.74,
23,Kapaa,US,86.00,few clouds,22.08,-159.32,
24,Vaini,TO,80.60,scattered clouds,-21.20,-175.20,
31,Dingle,PH,80.74,overcast clouds,11.00,122.67,
39,Butaritari,KI,82.60,overcast clouds,3.07,172.79,
42,Surt,LY,78.58,clear sky,31.21,16.59,
45,Mana,GF,78.66,broken clouds,5.66,-53.78,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ['Hotel Name'])

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Findlay,US,87.80,clear sky,41.04,-83.65,Drury Inn & Suites Findlay
9,Atuona,PF,78.82,broken clouds,-9.80,-139.03,Villa Enata
15,Marawi,PH,78.01,broken clouds,8.00,124.29,Salihmaruhom
19,Shouguang,CN,78.01,scattered clouds,36.88,118.74,Communications Hotel
23,Kapaa,US,86.00,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
686,Kilinochchi,LK,78.58,few clouds,9.40,80.40,Solaivanam Guest House
687,Lorengau,PG,84.69,light rain,-2.02,147.27,Lorengau Harbourside Hotel
688,Bubaque,GW,80.67,light rain,11.28,-15.83,Casa Dora
693,San Rafael Del Sur,NI,82.40,scattered clouds,11.85,-86.44,Hospedaje El Chino


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))